In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen

Reading in MSP data, using ZULU time variable to create a date and time stamp

In [2]:
HTM_url = urlopen("https://www.hackthemachine.ai/s/HTM_MSP_Finalcsv.zip")

zipfile = ZipFile(BytesIO(HTM_url.read()))

msp_data = pd.read_csv(zipfile.open('HTM_MSP_Final.csv'), parse_dates=True)

msp_data.rename(columns = {'AIRCRAFT': 'Aircraft'}, inplace = True)

msp_data['Fault Date'] = pd.to_datetime(msp_data['ZULU_TIME'].str.split(" ", n = 2, expand = True)[1])

In [3]:
msp_data.head()

,Aircraft,SQUADRON,LOT,MSP,ZULU_TIME,FLIGHT_MODE,Fault Date
0,1,HH,100,ZJMZTL,01-APR-2014 00:25:39:00,EngineTurn,2014-04-01
1,1,HH,100,JDJDMQ,01-APR-2014 00:25:39:00,EngineTurn,2014-04-01
2,1,HH,100,RTVBHP,01-APR-2014 00:25:39:00,EngineTurn,2014-04-01
3,1,HH,100,RTUXHP,01-APR-2014 00:25:39:00,EngineTurn,2014-04-01
4,1,HH,100,RTUXMZ,01-APR-2014 00:25:39:00,EngineTurn,2014-04-01


Reading in MAF data, converting recevied and completed dates to datetime objects

In [4]:
HTM_url = urlopen("https://www.hackthemachine.ai/s/HtM_MAF-Data_Finalcsv.zip")

zipfile = ZipFile(BytesIO(HTM_url.read()))

maf_data = pd.read_csv(zipfile.open('HtM_MAF Data_Final.csv'), parse_dates=['Received Date', 'Completion Date'])

In [5]:
maf_data.head()

,Job Code,Aircraft,Transaction Code,Malfunction Code,Action Taken Code,Description of Problem,Correction of Problem,Received Date,Completion Date,Corrosion,Bare Metal,Corrosion Prevention Treatment,Routine Maintenance,Unscheduled Maintenance,Mission-Related Maintenance,Failure
0,0NGHY44WC8118573,42,11,0,N,Perform system or component checks,Completed the component or system test,2012-04-26,2014-02-27,NaN,NaN,NaN,Yes,NaN,NaN,NaN
1,0NGHY45WC8118591,42,11,0,N,Perform system or component checks,Completed the component or system test,2012-04-26,2014-02-27,NaN,NaN,NaN,Yes,NaN,NaN,NaN
2,0NGHY46WC8118581,42,11,0,N,Perform a periodic inspection,Completed the inspection,2012-04-26,2014-02-27,NaN,NaN,NaN,Yes,NaN,NaN,NaN
3,0NGHY47WC8118577,42,11,0,N,Perform a periodic inspection,Completed the inspection,2012-04-26,2014-02-27,NaN,NaN,NaN,Yes,NaN,NaN,NaN
4,0NGHY48WC8118586,42,11,0,N,Perform a periodic inspection,Completed the inspection,2012-04-26,2014-02-27,NaN,NaN,NaN,Yes,NaN,NaN,NaN


Drop rountine mainentance

In [6]:
maf_data = maf_data[maf_data['Routine Maintenance'] != 'Yes']

Adding field that indicates whether repair pertained to corrosion, grouping by aircraft and received date

In [7]:
maf_data['corr_action'] = maf_data.groupby(['Aircraft', 'Received Date'])['Corrosion'].transform(lambda x: any(x == 'Yes'))

In [8]:
maf_data[['Aircraft', 'Received Date', 'corr_action']].drop_duplicates()['corr_action'].value_counts()

False    41521
True      2625
Name: corr_action, dtype: int64

based on the value counts of the corr_action variable, we have a highly imbalanced dataset.  We will use upsampling/downsampling and adjust our evaluation metric (precison, F1) to deal with this imbalance

Adding msp frequency to repair data

In [9]:
maf_corr = maf_data[['Aircraft', 'Received Date', 'Completion Date', 'corr_action']].drop_duplicates()

maf_corr.head()

,Aircraft,Received Date,Completion Date,corr_action
19,42,2012-04-26,2014-02-27,False
31,42,2012-04-26,2014-02-25,False
57,43,2012-09-13,2015-06-05,False
70,43,2012-09-13,2015-04-30,False
80,43,2012-09-14,2015-06-05,False


For each Aircraft repair, we will add the frequncy of the MSP codes in the 30 days prior

In [20]:
msp_corr_pre = []

for repair in maf_corr.iterrows():
    
    # matching aircraft
    
    msp_subset = msp_data[['Aircraft', 'MSP', 'Fault Date']][msp_data['Aircraft'] == repair[1][0]]
    
    # filtering out MSP codes outside of 30-day window prior to repair
    
    msp_subset = msp_subset[(repair[1][1] - msp_subset['Fault Date'] < np.timedelta64(30, 'D')) & (repair[1][1] - msp_subset['Fault Date'] > np.timedelta64(0, 'D'))]
    
    # adding repair date and corrosion flag
    
    msp_subset['Repair Date'] = repair[1][1]
    
    msp_subset['Corrosion'] = repair[1][3]
    
    # counting freq of each MSP code
    
    msp_counts = msp_subset.groupby(['Aircraft', 'Repair Date', 'MSP', 'Corrosion']).size().reset_index(name='freq')
    
    msp_corr_pre.append(msp_counts)

    
msp_corr_pre = pd.concat(msp_corr_pre)


In [11]:
msp_corr_pre = pd.read_csv('../../HTM_data/msp_freq_pre.csv')

FileNotFoundError: [Errno 2] File b'../../HTM_data/msp_freq_pre.csv' does not exist: b'../../HTM_data/msp_freq_pre.csv'

In [21]:
msp_corr_pre.head()

,Aircraft,Repair Date,MSP,Corrosion,freq
0,42.0,2012-04-26,FVFVJD,0.0,5
1,42.0,2012-04-26,FVFVZJ,0.0,18
2,42.0,2012-04-26,FVHPHP,0.0,111
3,42.0,2012-04-26,FVHPMZ,0.0,2
4,42.0,2012-04-26,FVHPZJ,0.0,2


Saving pre frequencies to CSV

In [68]:
msp_corr_pre.to_csv('../../HTM_data/msp_freq_pre.csv')

Convert Corrosion to T/F

In [22]:
msp_corr_pre['Corrosion'] = msp_corr_pre['Corrosion'] == 1.0

Convert Aircraft to Category

In [23]:
msp_corr_pre['Aircraft'] = msp_corr_pre['Aircraft'].astype('category')

Adding pre marker to frequency

In [38]:
msp_corr_pre.rename(columns = {'freq': 'pre_freq'}, inplace = True)

Based on SME input, the MSP codes we should focus on when building a predictive model typically stop appearing when an action for corrosion has been taken.  Therefore we will estimate mean frequncy before and after a corrosion repair for all MSP codes, and then focus the top 20 MSP codes with the largest decrease post-maintenance action.

First we will gather the frequncey of MSP codes 30-days post corrosion maintenance action, as we have already done so for the 30-day prio window.

In [ ]:
msp_corr_post = []

for repair in maf_corr.iterrows():
    
    # matching aircraft
    
    msp_subset = msp_data[['Aircraft', 'MSP', 'Fault Date']][msp_data['Aircraft'] == repair[1][0]]
    
    # filtering out MSP codes outside of 30-day window after Completion
    
    msp_subset = msp_subset[(repair[1][2] - msp_subset['Fault Date'] < np.timedelta64(30, 'D')) & (repair[1][2] - msp_subset['Fault Date'] < np.timedelta64(0, 'D'))]
    
    # adding repair date and corrosion flag
    
    msp_subset['Repair Date'] = repair[1][1]
    
    msp_subset['Corrosion'] = repair[1][3]
    
    # counting freq of each MSP code
    
    msp_counts = msp_subset.groupby(['Aircraft', 'Repair Date', 'MSP', 'Corrosion']).size().reset_index(name='freq')
    
    msp_corr_post.append(msp_counts)

    
msp_corr_post = pd.concat(msp_corr_post)

Saving post frequncies to CSV

In [31]:
msp_corr_post.to_csv('../../HTM_data/msp_freq_post.csv')

In [33]:
msp_corr_post.head()

,Aircraft,Repair Date,MSP,Corrosion,freq
0,42.0,2012-04-26,FVFVHP,0.0,1
1,42.0,2012-04-26,FVFVJD,0.0,13
2,42.0,2012-04-26,FVFVMQ,0.0,1
3,42.0,2012-04-26,FVFVXU,0.0,3
4,42.0,2012-04-26,FVFVZJ,0.0,266


Convert Corrosion to T/F

In [34]:
msp_corr_post['Corrosion'] = msp_corr_post['Corrosion'] == 1.0

Convert Aircraft to Category

In [35]:
msp_corr_post['Aircraft'] = msp_corr_post['Aircraft'].astype('category')

Adding post marker to frequency

In [36]:
msp_corr_post.rename(columns = {'freq': 'post_freq'}, inplace = True)

Merging pre/post frequencies

In [39]:
msp_corr_post.head(200).merge(msp_corr_pre, on = ['Aircraft', 'Repair Date', 'MSP', 'Corrosion'])

,Aircraft,Repair Date,MSP,Corrosion,post_freq,pre_freq
0,42.0,2012-04-26,FVFVJD,False,13,5
1,42.0,2012-04-26,FVFVZJ,False,266,18
2,42.0,2012-04-26,FVHPHP,False,3431,111
3,42.0,2012-04-26,FVHPMZ,False,8,2
4,42.0,2012-04-26,FVHPZJ,False,51,2
5,42.0,2012-04-26,FVMQHP,False,167,3
6,42.0,2012-04-26,FVMQMQ,False,31,1
7,42.0,2012-04-26,FVXUMQ,False,81,1
8,42.0,2012-04-26,HP,False,2411,101
9,42.0,2012-04-26,HPTLHP,False,5,3


In [129]:
msp_corr = msp_corr[msp_corr['MSP'].isin(msp_preds)]

In [276]:
msp_corr.head()

MSP,Aircraft,Repair Date,Corrosion,FV,FVFVJD,FVFVXU,FVHPHP,FVHPJD,FVMQVB,FVQG,...,UXMQNV,UXUXMQ,VBQGQG,XUFV,XUHPHP,XUXU,XUXUJD,XUXUVB,ZJMZBM,ZJMZTL
0,1.0,2013-09-24,False,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1.0,2013-10-03,False,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1.0,2013-10-04,False,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1.0,2013-10-16,False,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1.0,2013-10-17,False,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [161]:
msp_corr = msp_corr.drop_duplicates(['Aircraft', 'Repair Date', 'Corrosion'])

In [183]:
msp_corr = msp_corr.pivot_table(index = ['Aircraft', 'Repair Date', 'Corrosion'], columns = 'MSP', values = 'freq', fill_value = 0).reset_index()

In [277]:
msp_corr.head()

MSP,Aircraft,Repair Date,Corrosion,FV,FVFVJD,FVFVXU,FVHPHP,FVHPJD,FVMQVB,FVQG,...,UXMQNV,UXUXMQ,VBQGQG,XUFV,XUHPHP,XUXU,XUXUJD,XUXUVB,ZJMZBM,ZJMZTL
0,1.0,2013-09-24,False,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1.0,2013-10-03,False,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1.0,2013-10-04,False,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1.0,2013-10-16,False,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1.0,2013-10-17,False,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Save MSP count file to csv

In [189]:
msp_corr.to_csv('../../HTM_data/htm_predict.csv')

Selecting the first 35 aircrafts to train, last 10 to test

In [197]:
train_data = msp_corr[msp_corr['Aircraft'].isin(range(36))]

x_train = train_data.drop(['Aircraft', 'Repair Date', 'Corrosion'], axis=1)
y_train = train_data['Corrosion']

test_data = msp_corr[msp_corr['Aircraft'].isin(range(36,46))]

x_test = test_data.drop(['Aircraft', 'Repair Date', 'Corrosion'], axis=1)
y_test = test_data['Corrosion']

### Ridge Logistic Regression Model

In [290]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import confusion_matrix, accuracy_score

# ignore convergance warnings
from warnings import filterwarnings
filterwarnings('ignore')

Fitting model 

In [291]:
lasso = LogisticRegressionCV(penalty='l2', class_weight='balanced', cv = 5, max_iter=250).fit(x_train, y_train)

In [293]:
accuracy_score(y_test, lasso.predict(x_test))

0.5985009369144285

In [294]:
confusion_matrix(y_test, lasso.predict(x_test))

array([[4647, 3071],
       [ 143,  144]])

### XGBoost

### Random Forest

To account for unbalanced data: upsampling postive cases